In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

sns.set_style("whitegrid")
torch.manual_seed(0)
np.random.seed(0)

# Data Loading, Cleaning, EDA

In [59]:
train_path = "dataset/delhi-climate-data/DailyDelhiClimateTrain.csv"
test_path = "dataset/delhi-climate-data/DailyDelhiClimateTest.csv"
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

def clean_df(df):
    return (df
        .loc[:, ["date", "meantemp"]]
        .sort_values("date", ascending=True)
        .assign(
            date=lambda df_: pd.to_datetime(df_["date"]), 
            meantemp=lambda df_: df_["meantemp"].astype("float32")
        )
        .set_index("date")
        .resample("1d")
        .ffill()
        .reset_index()
    )

df_train = clean_df(df_train)
df_test = clean_df(df_test)

print(df_train.shape, df_test.shape)
df_train.head()

(1462, 2) (114, 2)


,date,meantemp
0,2013-01-01,10.000000
1,2013-01-02,7.400000
2,2013-01-03,7.166667
3,2013-01-04,8.666667
4,2013-01-05,6.000000


# Create Dataset

In [60]:
x_train = df_train["meantemp"].values[:-114]
x_val = df_train["meantemp"].values[-114:]
x_test = df_test["meantemp"].values

x_train_looped = []
for i in range(x_train.shape[0]):
    x = np.array(x_train[i : i + 13])
    if x.shape[0] == 13:
        x_train_looped.append(x.reshape(1, 13))
x_train_looped = np.array(x_train_looped).reshape(-1, 13)


x_val_looped = []
for i in range(x_val.shape[0]):
    x = np.array(x_val[i : i + 13])
    if x.shape[0] == 13:
        x_val_looped.append(x.reshape(1, 13))
x_val_looped = np.array(x_val_looped).reshape(-1, 13)


x_test_looped = []
for i in range(x_test.shape[0]):
    x = np.array(x_test[i : i + 13])
    if x.shape[0] == 13:
        x_test_looped.append(x.reshape(1, 13))
x_test_looped = np.array(x_test_looped).reshape(-1, 13)

print(x_train_looped.shape, x_val_looped.shape, x_test_looped.shape)

(1336, 13) (102, 13) (102, 13)


For training set, this is 1450 samples, each consisting of 13 months, etc for val and test set.

Next we need to create y from this.

In [61]:
x_train_tensor = torch.from_numpy(x_train_looped[:, :-1]).unsqueeze(dim=1)
x_val_tensor = torch.from_numpy(x_val_looped[:, :-1]).unsqueeze(dim=1)
x_test_tensor = torch.from_numpy(x_test_looped[:, :-1]).unsqueeze(dim=1)

y_train_tensor = torch.from_numpy(x_train_looped[:, -1]).unsqueeze(dim=1)
y_val_tensor = torch.from_numpy(x_val_looped[:, -1]).unsqueeze(dim=1)
y_test_tensor = torch.from_numpy(x_test_looped[:, -1]).unsqueeze(dim=1)

print(x_train_tensor.shape, x_val_tensor.shape, x_test_tensor.shape)
print(y_train_tensor.shape, y_val_tensor.shape, y_test_tensor.shape)

torch.Size([1336, 1, 12]) torch.Size([102, 1, 12]) torch.Size([102, 1, 12])
torch.Size([1336, 1]) torch.Size([102, 1]) torch.Size([102, 1])


In [62]:
x_train_tensor

tensor([[[10.0000,  7.4000,  7.1667,  ..., 11.0000, 15.7143, 14.0000]],

        [[ 7.4000,  7.1667,  8.6667,  ..., 15.7143, 14.0000, 15.8333]],

        [[ 7.1667,  8.6667,  6.0000,  ..., 14.0000, 15.8333, 12.8333]],

        ...,

        [[31.2222, 31.7857, 33.4000,  ..., 31.6923, 31.0769, 30.3750]],

        [[31.7857, 33.4000, 29.5714,  ..., 31.0769, 30.3750, 31.1000]],

        [[33.4000, 29.5714, 30.0400,  ..., 30.3750, 31.1000, 31.9167]]])

In [63]:
y_train_tensor

tensor([[15.8333],
        [12.8333],
        [14.7143],
        ...,
        [31.1000],
        [31.9167],
        [30.5556]])

# Model Architecture

In [64]:
class LSTM(nn.Module):
    def __init__(self, hidden_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm1 = nn.LSTMCell(input_size=1, hidden_size=hidden_size)
        self.lstm2 = nn.LSTMCell(input_size=hidden_size, hidden_size=hidden_size)
        self.linear = nn.Linear(in_features=hidden_size, out_features=1)

    def forward(self, xs, future=1):
        outputs = []
        h1 = torch.zeros(xs.shape[0], self.hidden_size)
        c1 = torch.zeros(xs.shape[0], self.hidden_size)
        h2 = torch.zeros(xs.shape[0], self.hidden_size)
        c2 = torch.zeros(xs.shape[0], self.hidden_size)
        # iterate through each value of all samples toegther
        for x in xs.split(1, dim=1):
            h1, c1 = self.lstm1(x, (h1, c1))
            h2, c2 = self.lstm2(h1, (h2, c2))
            output = self.linear(h2)
            outputs += [output]
        # predict future values
        for i in range(future):
            h1, c1 = self.lstm1(output, (h1, c1))
            h2, c2 = self.lstm2(h1, (h2, c2))
            output = self.linear(h2)
            outputs += [output]

        outputs = torch.cat(outputs, dim=1)
        # only return the last value
        return outputs[:, -1]
    
lstm = LSTM(hidden_size=64)

# test if our model is working
samples = torch.zeros((3, 12))
lstm(samples)

tensor([-0.0989, -0.0989, -0.0989], grad_fn=<SelectBackward0>)

# Training

In [65]:
criterion = nn.MSELoss()
lr = 0.01
optimizer = optim.Adam(lstm.parameters(), lr=lr)
epochs = 10
print_step = 2

for epoch in range(epochs):
    train_loss = []
    val_loss = []

    # training
    for x, y in zip(x_train_tensor, y_train_tensor):

        lstm.train()
        optimizer.zero_grad()
        outputs = lstm(x)
        loss = criterion(y, outputs)
        loss.backward()
        optimizer.step()
        train_loss += [np.sqrt(loss.item())]

    # validation
    for x, y in zip(x_val_tensor, y_val_tensor):

        lstm.eval()
        with torch.no_grad():
            outputs = lstm(x)
            loss = criterion(y, outputs)
            val_loss += [np.sqrt(loss.item())]

    if (epoch == 0) or ((epoch + 1) % print_step == 0):
        # print train and validation result
        avg_train_loss = np.mean(train_loss)
        avg_val_loss = np.mean(val_loss)
        print(
            f"Epoch {epoch+1: <3}/{epochs} | train RMSE = {avg_train_loss: .8f} | val RMSE = {avg_val_loss: .8f}"
        )

Epoch 1  /10 | train RMSE =  2.05381160 | val RMSE =  6.97248867
Epoch 2  /10 | train RMSE =  2.11188664 | val RMSE =  6.98830486
Epoch 4  /10 | train RMSE =  2.11265611 | val RMSE =  6.99226452
Epoch 6  /10 | train RMSE =  2.11268858 | val RMSE =  6.99252129
Epoch 8  /10 | train RMSE =  2.10330934 | val RMSE =  7.07701865
Epoch 10 /10 | train RMSE =  2.10910573 | val RMSE =  7.07848731


# Test

In [67]:
outputs = []

for x, y in zip(x_test_tensor, y_test_tensor):

    lstm.eval()
    with torch.no_grad():
        output = lstm(x, future=1)
        # output = output[0, -1].item()
        # outputs.append(output)

    print(y)
    print(output)
    break

tensor([13.2353])
tensor([30.5397])
